<a href="https://colab.research.google.com/github/Web-Jose/CSCI-164/blob/main/AI24Ch3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import random
import heapq

# Tile Sliding Domain: Initial State Space

In [64]:
StateDimension=3
InitialState = [1,2,3,4,5,6,0,7,8]
GoalState=[1,2,3,4,5,6,7,8,0]
Actions = lambda s: ['u', 'd', 'l', 'r']
Opposite=dict([('u','d'),('d','u'),('l','r'),('r','l'), (None, None)])

In [65]:
def Result(state, action):
  i = state.index(0)
  newState = list(state)
  row,col=i//StateDimension, i % StateDimension
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return newState
  if action=='u':
    l,r = row*StateDimension+col, (row-1)*StateDimension+col
  elif action=='d':
    l,r = row*StateDimension+col, (row+1)*StateDimension+col
  elif action=='l':
    l,r = row*StateDimension+col, row*StateDimension+col-1
  elif action=='r' :
    l,r = row*StateDimension+col, row*StateDimension+col+1
  newState[l], newState[r] = newState[r], newState[l]
  return newState

def PrintState(s):
  for i in range(0,len(s),StateDimension):
    print(s[i:i+StateDimension])

def LegalMove(state, action):
  i = state.index(0)
  row,col=i//StateDimension, i % StateDimension
  newState = state.copy()
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return False
  return True


In [66]:
def SingleTileManhattanDistance(tile, left, right):
  leftIndex = left.index(tile)
  rightIndex = right.index(tile)
  return (abs(leftIndex//StateDimension-rightIndex//StateDimension) +
          abs(leftIndex%StateDimension-rightIndex%StateDimension))

def ManhattanDistance(left, right):
  distances = [SingleTileManhattanDistance(tile, left, right)
     for tile in range(1, StateDimension**2)]
  ### print ("Distances= ", distances)
  return sum(distances)


In [68]:
PrintState(InitialState)

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]


In [69]:
PrintState(GoalState)

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]


In [70]:
print("ManhattanDistance=  ", ManhattanDistance(InitialState, GoalState))
print("OutOfPlace= ", OutOfPlace(InitialState, GoalState))


ManhattanDistance=   2
OutOfPlace=  2


In [74]:
def ApplyMoves(actions, state):
  for action in actions:
    state = Result(state, action)
  return state

In [ ]:
def Solution(node):
    return node.solution()

def BreadthFirstSearch(problem):
    from collections import deque
    frontier = deque([Node(problem.INITIAL)])
    explored = set()
    while frontier:
        node = frontier.popleft()
        if problem.IsGoal(node.State):
            return node, len(explored)
        explored.add(tuple(node.State))
        for action in problem.Actions(node.State):
            child = node.ChildNode(problem, action)
            if tuple(child.State) not in explored and child not in frontier:
                frontier.append(child)
    return None, len(explored)

## Node

In [79]:
class Node(object):
  def __init__(self, state, parent=None, action=None, path_cost=0 ):
    self.State=state
    self.Parent=parent
    self.Action=action
    self.PathCost = path_cost

  def __str__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __repr__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __lt__(self, other):
    return self.PathCost < other.PathCost;

## Breadth-First Search

In [81]:
def BreadthFirstSearch(problem):
  node = Node(tuple(problem.INITIAL))
  if problem.IsGoal(node.State):
    return node, 0
  Frontier = []
  Frontier.append(node)
  reached = set()
  reached.add(tuple(problem.INITIAL))
  nodesExpanded = 0
  while (Frontier):
    ### print([str(n) for n in Frontier])
    node = Frontier.pop(0)
    ### print(node)
    for child in Expand(problem, node):
      s = tuple(child.State)
      ### print (s, "IsGoal=", problem.IsGoal(s))
      if problem.IsGoal(s):
        return child, nodesExpanded
      if s not in reached:
        reached.add(s)
        Frontier.append(child)
    nodesExpanded += 1
    if nodesExpanded > 500000:
      break;
  return None, nodesExpanded

## Best-First Search

In [82]:
def BestFirstSearch(problem, f):
  node = Node(tuple(problem.INITIAL))
  Frontier = []
  heapq.heappush(Frontier,(f(node), node))
  reached = {}
  reached[tuple(problem.INITIAL)]=node
  nodesExpanded = 0
  while (Frontier):
    ##print([(x, str(n)) for (x,n) in Frontier])
    fValue, node = heapq.heappop(Frontier)
    ##print (node.State, "IsGoal=", problem.IsGoal(tuple(node.State)))
    if problem.IsGoal(tuple(node.State)):
      return node, nodesExpanded    ### print(node)
    for child in Expand(problem, node):
      s = tuple(child.State)
      if s not in reached or child.PathCost < reached[s].PathCost:
        reached[s] = child
        heapq.heappush(Frontier, (f(child), child))
    nodesExpanded += 1
    if nodesExpanded > 500000:
      break;
  return None, nodesExpanded

## Problem 1

In [83]:
TileSliding = Problem()
TileSliding.INITIAL = InitialState
TileSliding.IsGoal = lambda s: s==(1,2,3,4,5,6,7,8,0)
TileSliding.Actions = Actions
TileSliding.Result=Result
TileSliding.ActionCost = lambda s, a, sPrime: 1
print( TileSliding.IsGoal((1,2,3,4,5,6,7,8,0)) )
print( Node(InitialState) )
print(1+TileSliding.ActionCost(1,2,3))

True
[1, 2, 3, 4, 5, 6, 0, 7, 8], <none>
2


In [84]:
TileSliding.INITIAL = [1,2,3,4,5,6,0,7,8]
ret, cost = BreadthFirstSearch(TileSliding)
print (ret, cost)

[1, 2, 3, 4, 5, 6, 7, 8, 0], r 2


In [85]:
def Solution(node):
  if node.Parent==None:
    return []
  return Solution(node.Parent) + [node.Action]


In [86]:
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))

['r', 'r']
[1, 2, 3, 4, 5, 6, 0, 7, 8]
[1, 2, 3, 4, 5, 6, 7, 8, 0]


In [87]:
TileSliding.INITIAL = [1,2,3,4,0,6,7,5,8]
ret, cost = BreadthFirstSearch(TileSliding)
print (ret, cost)

[1, 2, 3, 4, 5, 6, 7, 8, 0], r 2


In [88]:
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))

['d', 'r']
[1, 2, 3, 4, 0, 6, 7, 5, 8]
[1, 2, 3, 4, 5, 6, 7, 8, 0]


In [89]:
UniformCostF = lambda n: n.PathCost
AStarF = lambda n: n.PathCost+ManhattanDistance(n.State, GoalState)
TileSliding.INITIAL = [1,2,3,4,0,6,7,5,8]
ret, cost = BestFirstSearch(TileSliding, UniformCostF)
print (ret)
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))
print ("Nodes Expanded=", cost)

[1, 2, 3, 4, 5, 6, 7, 8, 0], r
['d', 'r']
[1, 2, 3, 4, 0, 6, 7, 5, 8]
[1, 2, 3, 4, 5, 6, 7, 8, 0]
Nodes Expanded= 8


# Problem 2

In [90]:
state1, sol = RandomWalk(GoalState, 300)
PrintState(state1)
print (sol)
print(ReverseMoves(sol))
PrintState (ApplyMoves(ReverseMoves(sol), state1))

[1, 4, 8]
[3, 0, 6]
[7, 2, 5]
['l', 'u', 'l', 'u', 'r', 'r', 'd', 'd', 'l', 'l', 'u', 'u', 'r', 'd', 'r', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'd', 'r', 'd', 'l', 'l', 'u', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'd', 'r', 'r', 'u', 'u', 'l', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'u', 'r', 'r', 'd', 'l', 'd', 'r', 'u', 'l', 'u', 'r', 'd', 'l', 'u', 'r', 'd', 'd', 'l', 'l', 'u', 'r', 'd', 'l', 'u', 'r', 'u', 'r', 'd', 'd', 'l', 'l', 'u', 'r', 'u', 'l', 'd', 'r', 'u', 'l', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'd', 'r', 'u', 'r', 'u', 'l', 'l', 'd', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'u', 'r', 'u', 'r', 'd', 'd', 'l', 'u', 'l', 'u', 'r', 'r', 'd', 'd', 'l', 'l', 'u', 'u', 'r', 'd', 'r', 'd', 'l', 'u', 'r', 'u', 'l', 'l', 'd', 'd', 'r', 'r', 'u', 'l', 'u', 'l', 'd', 'r', 'u', 'l', 'd', 'r', 'u', 'l', 'd', 'r', 'd', 'l', 'u', 'r', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'd', 'l', 'u', 'r', 'd', 'r', 'u', 'l', 'd', 'r', 'u', 'l', 'd', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'd',

In [91]:
TileSliding.INITIAL = state1
ret, cost = BreadthFirstSearch(TileSliding)
print (ret)
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))
print ("Length of solution: ", len(sol))
print ("Nodes Expanded=", cost)

[1, 2, 3, 4, 5, 6, 7, 8, 0], r
['r', 'u', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'l', 'u', 'r', 'd', 'r', 'u', 'l', 'l', 'd', 'r', 'd', 'r']
[1, 4, 8, 3, 0, 6, 7, 2, 5]
[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  22
Nodes Expanded= 78750


In [92]:
UniformCostF = lambda n: n.PathCost
TileSliding.INITIAL = state1
ret, cost = BestFirstSearch(TileSliding, UniformCostF)
print (ret)
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))
print ("Length of solution: ", len(sol))
print ("Nodes Expanded=", cost)

[1, 2, 3, 4, 5, 6, 7, 8, 0], r
['r', 'u', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'l', 'u', 'r', 'd', 'r', 'u', 'l', 'l', 'd', 'r', 'd', 'r']
[1, 4, 8, 3, 0, 6, 7, 2, 5]
[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  22
Nodes Expanded= 87177


# Problem List

In [93]:
findNum = 10
randomWalkDistance = 300
problemList = []
for i in range(10):
  state1, sol = RandomWalk(GoalState, 300)
  problemList.append(state1)
print (problemList)

[[5, 7, 2, 3, 0, 6, 4, 1, 8], [0, 3, 1, 2, 5, 8, 4, 6, 7], [3, 8, 5, 2, 0, 7, 4, 1, 6], [8, 3, 1, 4, 6, 7, 0, 2, 5], [0, 2, 8, 7, 3, 6, 5, 1, 4], [3, 2, 8, 4, 0, 1, 7, 6, 5], [3, 6, 8, 5, 7, 1, 0, 4, 2], [4, 2, 5, 8, 7, 6, 0, 3, 1], [3, 7, 4, 2, 8, 5, 0, 1, 6], [3, 8, 5, 2, 4, 1, 0, 7, 6]]


### Breadth First Search w/ Test Problems

In [94]:
testProblems = [[5, 3, 0, 4, 7, 6, 2, 1, 8], [3, 2, 0, 1, 5, 4, 7, 8, 6], [0, 1, 3, 8, 5, 2, 6, 7, 4],
                [3, 7, 6, 2, 4, 1, 8, 5, 0], [6, 7, 5, 8, 0, 1, 2, 3, 4], [7, 4, 6, 8, 0, 1, 3, 2, 5],
                [0, 1, 8, 3, 5, 2, 6, 4, 7], [0, 4, 6, 5, 8, 1, 7, 2, 3], [1, 5, 7, 6, 0, 3, 4, 2, 8],
                [0, 2, 4, 1, 7, 6, 3, 5, 8]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BreadthFirstSearch(TileSliding)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)


['l', 'd', 'l', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'r', 'r']
-----------------------
[5, 3, 0, 4, 7, 6, 2, 1, 8] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  18
Nodes Expanded= 15775
-----------------------
['l', 'd', 'r', 'u', 'l', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'r', 'd']
-----------------------
[3, 2, 0, 1, 5, 4, 7, 8, 6] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  14
Nodes Expanded= 2645
-----------------------
['d', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'u', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'u', 'r', 'r', 'd']
-----------------------
[0, 1, 3, 8, 5, 2, 6, 7, 4] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  24
Nodes Expanded= 99562
-----------------------
['u', 'l', 'u', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'd', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'r', 'r', 'd']
-----------------------
[3, 7, 6, 2, 4, 1, 8, 5, 0] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  24
Nodes Expanded= 102839
------------

### Uniform Cost Search w/ Test Problems

In [95]:
UniformCostF = lambda n: n.PathCost
testProblems = [[5, 3, 0, 4, 7, 6, 2, 1, 8], [3, 2, 0, 1, 5, 4, 7, 8, 6], [0, 1, 3, 8, 5, 2, 6, 7, 4],
                [3, 7, 6, 2, 4, 1, 8, 5, 0], [6, 7, 5, 8, 0, 1, 2, 3, 4], [7, 4, 6, 8, 0, 1, 3, 2, 5],
                [0, 1, 8, 3, 5, 2, 6, 4, 7], [0, 4, 6, 5, 8, 1, 7, 2, 3], [1, 5, 7, 6, 0, 3, 4, 2, 8],
                [0, 2, 4, 1, 7, 6, 3, 5, 8]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, UniformCostF)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

['l', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'r', 'd', 'r']
-----------------------
[5, 3, 0, 4, 7, 6, 2, 1, 8] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  18
Nodes Expanded= 22919
-----------------------
['l', 'd', 'r', 'u', 'l', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'r', 'd']
-----------------------
[3, 2, 0, 1, 5, 4, 7, 8, 6] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  14
Nodes Expanded= 4754
-----------------------
['r', 'd', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'u', 'u', 'r', 'd', 'r', 'u', 'l', 'l', 'd', 'r', 'u', 'r', 'd', 'd']
-----------------------
[0, 1, 3, 8, 5, 2, 6, 7, 4] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  24
Nodes Expanded= 120796
-----------------------
['u', 'l', 'u', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'd', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'r', 'r', 'd']
-----------------------
[3, 7, 6, 2, 4, 1, 8, 5, 0] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  24
Nodes Expanded= 121538
-----------

### AStar using ManhattanDistance w/ Test Problems

In [96]:
AStarFb = lambda n: n.PathCost + Manhattan(n.State, GoalState)
testProblems = [[5, 3, 0, 4, 7, 6, 2, 1, 8], [3, 2, 0, 1, 5, 4, 7, 8, 6], [0, 1, 3, 8, 5, 2, 6, 7, 4],
                [3, 7, 6, 2, 4, 1, 8, 5, 0], [6, 7, 5, 8, 0, 1, 2, 3, 4], [7, 4, 6, 8, 0, 1, 3, 2, 5],
                [0, 1, 8, 3, 5, 2, 6, 4, 7], [0, 4, 6, 5, 8, 1, 7, 2, 3], [1, 5, 7, 6, 0, 3, 4, 2, 8],
                [0, 2, 4, 1, 7, 6, 3, 5, 8]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, AStarF)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

['l', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'r', 'd', 'r']
-----------------------
[5, 3, 0, 4, 7, 6, 2, 1, 8] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  18
Nodes Expanded= 251
-----------------------
['l', 'd', 'r', 'u', 'l', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'r', 'd']
-----------------------
[3, 2, 0, 1, 5, 4, 7, 8, 6] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  14
Nodes Expanded= 92
-----------------------
['d', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'u', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'u', 'r', 'r', 'd']
-----------------------
[0, 1, 3, 8, 5, 2, 6, 7, 4] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  24
Nodes Expanded= 2798
-----------------------
['u', 'l', 'u', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'd', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'r', 'r', 'd']
-----------------------
[3, 7, 6, 2, 4, 1, 8, 5, 0] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  24
Nodes Expanded= 1769
-------------------

### AStar using OutOfPlace w/ Test Problems

In [97]:
AStarFb = lambda n: n.PathCost + OutOfPlace(n.State, GoalState)
testProblems = [[5, 3, 0, 4, 7, 6, 2, 1, 8], [3, 2, 0, 1, 5, 4, 7, 8, 6], [0, 1, 3, 8, 5, 2, 6, 7, 4],
                [3, 7, 6, 2, 4, 1, 8, 5, 0], [6, 7, 5, 8, 0, 1, 2, 3, 4], [7, 4, 6, 8, 0, 1, 3, 2, 5],
                [0, 1, 8, 3, 5, 2, 6, 4, 7], [0, 4, 6, 5, 8, 1, 7, 2, 3], [1, 5, 7, 6, 0, 3, 4, 2, 8],
                [0, 2, 4, 1, 7, 6, 3, 5, 8]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, AStarFb)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

['l', 'd', 'l', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'r', 'r']
-----------------------
[5, 3, 0, 4, 7, 6, 2, 1, 8] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  18
Nodes Expanded= 1319
-----------------------
['l', 'd', 'r', 'u', 'l', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'r', 'd']
-----------------------
[3, 2, 0, 1, 5, 4, 7, 8, 6] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  14
Nodes Expanded= 242
-----------------------
['d', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'u', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'u', 'r', 'r', 'd']
-----------------------
[0, 1, 3, 8, 5, 2, 6, 7, 4] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  24
Nodes Expanded= 18751
-----------------------
['u', 'l', 'u', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'd', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'r', 'r', 'd']
-----------------------
[3, 7, 6, 2, 4, 1, 8, 5, 0] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  24
Nodes Expanded= 17913
---------------

### Best First Search -- Greedy

In [98]:
### Best First
bestFirstSearchf = lambda n: OutOfPlace(n.State, GoalState)
testProblems = [[5, 3, 0, 4, 7, 6, 2, 1, 8], [3, 2, 0, 1, 5, 4, 7, 8, 6], [0, 1, 3, 8, 5, 2, 6, 7, 4],
                [3, 7, 6, 2, 4, 1, 8, 5, 0], [6, 7, 5, 8, 0, 1, 2, 3, 4], [7, 4, 6, 8, 0, 1, 3, 2, 5],
                [0, 1, 8, 3, 5, 2, 6, 4, 7], [0, 4, 6, 5, 8, 1, 7, 2, 3], [1, 5, 7, 6, 0, 3, 4, 2, 8],
                [0, 2, 4, 1, 7, 6, 3, 5, 8]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, bestFirstSearchf)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

['l', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'r', 'd', 'r']
-----------------------
[5, 3, 0, 4, 7, 6, 2, 1, 8] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  18
Nodes Expanded= 52
-----------------------
['l', 'l', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'u', 'r', 'r', 'd', 'l', 'u', 'l', 'd', 'r', 'r', 'd']
-----------------------
[3, 2, 0, 1, 5, 4, 7, 8, 6] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  20
Nodes Expanded= 196
-----------------------
['r', 'd', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'u', 'r', 'u', 'l', 'd', 'd', 'r', 'u', 'l', 'u', 'r', 'd', 'd', 'r']
-----------------------
[0, 1, 3, 8, 5, 2, 6, 7, 4] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  30
Nodes Expanded= 380
-----------------------
['u', 'u', 'l', 'l', 'd', 'r', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'r', 'r', 'd', 'l', 'u', 'l', 'd', 'r', 'r']
-----------------------
[3, 7, 6, 2, 4, 1, 8, 5, 0] 

[1, 2, 3, 4, 

In [99]:
### Best First
bestFirstSearchf = lambda n: ManhattanDistance(n.State, GoalState)
testProblems = [[5, 3, 0, 4, 7, 6, 2, 1, 8], [3, 2, 0, 1, 5, 4, 7, 8, 6], [0, 1, 3, 8, 5, 2, 6, 7, 4],
                [3, 7, 6, 2, 4, 1, 8, 5, 0], [6, 7, 5, 8, 0, 1, 2, 3, 4], [7, 4, 6, 8, 0, 1, 3, 2, 5],
                [0, 1, 8, 3, 5, 2, 6, 4, 7], [0, 4, 6, 5, 8, 1, 7, 2, 3], [1, 5, 7, 6, 0, 3, 4, 2, 8],
                [0, 2, 4, 1, 7, 6, 3, 5, 8]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, bestFirstSearchf)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

['l', 'l', 'd', 'd', 'r', 'u', 'l', 'd', 'r', 'u', 'l', 'u', 'r', 'd', 'd', 'l', 'u', 'u', 'r', 'd', 'l', 'd', 'r', 'r']
-----------------------
[5, 3, 0, 4, 7, 6, 2, 1, 8] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  24
Nodes Expanded= 263
-----------------------
['l', 'l', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'u', 'r', 'r', 'd', 'l', 'u', 'l', 'd', 'r', 'r', 'd']
-----------------------
[3, 2, 0, 1, 5, 4, 7, 8, 6] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  20
Nodes Expanded= 193
-----------------------
['r', 'd', 'l', 'd', 'r', 'u', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'd', 'r', 'u', 'u', 'l', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'u', 'l', 'd', 'r', 'r', 'd']
-----------------------
[0, 1, 3, 8, 5, 2, 6, 7, 4] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  46
Nodes Expanded= 412
-----------------------
['u', 'u', 'l', 'l', 'd', 'r', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'l', 'u', 'r', 'u', 'l', 'd', '

## Problem Setup

We define the problem space as a `VectorWorld` with dimensions and actions that the agent can take. The goal state is defined as reaching a particular position in this vector space.

In [100]:
VectorWorldDim = 10
VectorWorld = Problem()
VectorWorld.INITIAL = [0]
VectorWorld.IsGoal = lambda s: s==[3,] or s==(3,)
VectorWorld.Actions = lambda s: ['Left', 'Right']
## TileSliding.Result=VectorWorldResult
VectorWorld.ActionCost = lambda s, a, sPrime: 1

## Defining the Result Function

The `VectorWorldResult` function determines the result of an action in the vector space. The agent can move either left or right, and the state updates accordingly.

In [101]:
def VectorWorldResult(state, action):
  if action=='Left':
    return [(state[0]+VectorWorldDim-1)%VectorWorldDim]
  else:
    return [(state[0]+1)%VectorWorldDim]
VectorWorld.Result=VectorWorldResult


### Goal Check

Here, we check if a particular state is the goal state, which is essential for our pathfinding logic to determine if the solution has been reached.

In [102]:
print (VectorWorld.IsGoal((3,)))

True


In [103]:
ret, cost = BreadthFirstSearch(VectorWorld)
print ("ret=", ret)
sol = Solution(ret)
print (sol)


ret= [3], Right
['Right', 'Right', 'Right']


In [104]:
VectorWorld.INITIAL = [8]

In [105]:
ret, cost = BreadthFirstSearch(VectorWorld)
print ("ret=", ret)
sol = Solution(ret)
print (sol)

ret= [3], Left
['Left', 'Left', 'Left', 'Left', 'Left']
